In [1]:
# materialDatabase

function liquidWater()
    name = "liquidwater"
    gamma = 2.8
    pinf = 8.5e8
    cp = 4186.
    return name, gamma, pinf, cp
end

function air()
    name = "air"
    gamma = 1.4
    pinf = 0.0
    cp = 1005.
    return name, gamma, pinf, cp
end

function default()
    name = "default"
    gamma = 1.4
    pinf = 0.0
    cp = gamma/(gamma-1.)*8.314462618
    return name, gamma, pinf, cp
end

# interface of the database:

    # get name of a material
    function getName( a_material )
        return a_material[1]
    end

    # get gamma of a material
    function getGamma( a_material )
        return a_material[2]
    end

    # get pinf of a material
    function getPinf( a_material )
        return a_material[3]
    end

    # get cp of a material
    function getCp( a_material )
        return a_material[4]
    end

    function materialDatabase()
        return liquidWater(), air(), default()
    end

    function getNames(materialDatabase=materialDatabase)
        return collect( getName(material) for material in materialDatabase() )
    end

    function getGammas(materialDatabase=materialDatabase)
        return collect( getGamma(material) for material in materialDatabase() )
    end

    function getPinfs(materialDatabase=materialDatabase)
        return collect( getPinf(material) for material in materialDatabase() )
    end

    function getCps(materialDatabase=materialDatabase)
        return collect( getCps(material) for material in materialDatabase() )
    end

    function getMaterialNamed(a_name::String)
        if ( eltype(materialDatabase()[ findall(x->x==a_name, getNames()) ])==Union{})
            throw(DomainError(a_name, "material not in database"))
        end
        return materialDatabase()[ findall(x->x==a_name, getNames() )[1] ]
    end

getMaterialNamed (generic function with 1 method)

In [2]:
function checkUnknown(desiredQuantity::String)
        listofvariables = ["pressure", "density", "temperature"]
    if ( !(desiredQuantity in listofvariables))
        throw(DomainError(desiredQuantity, "desiredQuantity must be either density, pressure or temperature"))
    else
        println("desired quantity: " , desiredQuantity)
    end
    return nothing
end

checkUnknown (generic function with 1 method)

In [3]:
function initializeRho_given_desiredQuantity!(givenQuantity2::Float64, desiredQuantity)
    list = ["pressure", "temperature"]
    if( desiredQuantity in list)
        return givenQuantity2
    elseif( desiredQuantity=="density")
        return nothing # will be calculated later
    else
        throw(DomainError(desiredQuantity, "unexpected desiredQuantity in initializeRho_given_desiredQuantity!"))
    end
end

initializeRho_given_desiredQuantity! (generic function with 1 method)

In [4]:
function initializeT_given_desiredQuantity!(givenQuantity1::Float64, desiredQuantity)
    list = ["pressure", "density"]
    if( desiredQuantity in list)
        return givenQuantity1
    elseif( desiredQuantity=="temperature")
        return nothing # will be calculated later
    else
        throw(DomainError(desiredQuantity, "unexpected desiredQuantity in initializeT_given_desiredQuantity!"))
    end
end

initializeT_given_desiredQuantity! (generic function with 1 method)

In [5]:
function initializeP_given_desiredQuantity!(givenQuantity1::Float64, desiredQuantity)
    list = ["temperature", "density"]
    if( desiredQuantity in list)
        return givenQuantity1
    elseif( desiredQuantity=="pressure")
        return nothing # will be calculated later
    else
        throw(DomainError(desiredQuantity, "unexpected desiredQuantity in initializeP_given_desiredQuantity!"))
    end
end

initializeP_given_desiredQuantity! (generic function with 1 method)

In [6]:
function initializeVariables_p_T_rho(givenQuantity1=givenQuantity1, givenQuantity2=givenQuantity2, desiredQuantity=desiredQuantity)
    # desiredQuantity -> givenQuantity1, givenQuantity2
    #       p         ->        T      ,      rho
    #       T         ->        p      ,      rho
    #       rho       ->        T      ,      p
    
    rho = initializeRho_given_desiredQuantity!(givenQuantity2,desiredQuantity);
    T = initializeT_given_desiredQuantity!(givenQuantity1, desiredQuantity);
    p = initializeP_given_desiredQuantity!(givenQuantity1 ,desiredQuantity);
    if(desiredQuantity=="density")
        p = initializeP_given_desiredQuantity!(givenQuantity2 ,desiredQuantity);
    end
    
    return p, T, rho
end

initializeVariables_p_T_rho (generic function with 4 methods)

In [7]:
function createMaterial(materialName::String, materialGamma=nothing, materialPinf=nothing, materialCp=nothing, materialDatabase=materialDatabase)
    
    name = materialName
    gamma = materialGamma
    pinf = materialPinf
    cp = materialCp
    
    if ( !(materialName in getNames(materialDatabase)) && (materialGamma==nothing || materialPinf==nothing || materialCp==nothing) )
        throw(DomainError(name, "material not in database: you need to specify gamma and pinf in main()"))
    elseif (!(materialName in getNames(materialDatabase) )   )
        println("custom material named <<", name, ">>, with gamma = ", gamma, ", pinf = ", pinf, ", and cp = ", cp)
    elseif ( materialName in getNames(materialDatabase) )
        if(materialGamma==nothing && materialPinf==nothing && materialCp==nothing)
            if ( materialName=="default" )
                default = getMaterialNamed("default")
                gamma = getGamma(default)
                pinf = getPinf(default)
                cp = getCp(default)
                println( "default material with gamma = ", gamma, ", pinf = ", pinf, " , and cp = ", cp)
            else
                material = getMaterialNamed(materialName)
                println("selected material <<", getName(material), ">> found in materialDatabase. Its properties are")
                name = getName(material)
                println("gamma = ", getGamma(material))
                gamma = getGamma(material)
                println("pinf = ", getPinf(material))
                pinf = getPinf(material)
                println("cp = ", getCp(material))
                cp = getCp(material)
            end
        else
            material = getMaterialNamed(materialName)
            if(gamma==nothing)
                gamma= getGamma(material)
            end
            if(pinf==nothing)
                pinf= getPinf(material)
            end
            if(cp==nothing)
                cp= getCp(material)
            end
            println("WARNING: using <<", materialName, ">> with new properties:")
            println("gamma = ", gamma)
            println("pinf = ", pinf)
        end  # second elseif
        
    end  # if
    
    the_used_material = (name, gamma, pinf, cp)    
    
    return the_used_material
end  # function

createMaterial (generic function with 5 methods)

In [8]:
function calculateDesiredQuantity!( p, T, rho, desiredQuantity, material)
    
    gamma = getGamma(material)
    pinf = getPinf(material)
    cp = getCp(material)
    
    R = ((gamma - 1.)/gamma)*cp   # specific gas constant (i.e., per unit mass)
    
    if(desiredQuantity=="temperature")
        T = (p + pinf)/rho/R
    end
    
    if(desiredQuantity=="pressure")
        p = rho*(gamma-1)*cp*T/gamma - pinf
    end
    
    if(desiredQuantity=="density")
        rho = (p + pinf)/T/R
    end
    
    e = cp*T/gamma + pinf/rho     # internal energy
    
    return p, T, rho, e
end

calculateDesiredQuantity! (generic function with 1 method)

In [9]:
function main(desiredQuantity::String, givenQuantity1, givenQuantity2, materialName="default", gamma=nothing, pinf=nothing, cp=nothing, materialDatabase=materialDatabase)
    
    # cast to Float64:
    givenQuantity1 = Float64(givenQuantity1)
    givenQuantity2 = Float64(givenQuantity2)
    
    # convert to lower case desiredQuantity
    desiredQuantity = lowercase(desiredQuantity)
    
    # convert to lower case materialName
    materialName = lowercase(materialName)
    
    # perform a few validity checks and initialize material
    checkUnknown(desiredQuantity)
    println()
    material = createMaterial(materialName, gamma, pinf, cp)
    
    # initialize pressure, temperature and density
    (p, T, rho) = initializeVariables_p_T_rho(givenQuantity1, givenQuantity2, desiredQuantity)
    
    # run the calculation of the desired quantity
    (p, T, rho, e) = calculateDesiredQuantity!(p, T, rho, desiredQuantity, material)
    
    # output
    println()
    println("output: ")
    println("p = ", p)
    println("T = ", T)
    println("rho = ", rho)
    println("e = ", e)
    println()
    println("ending main")
                
end

main (generic function with 6 methods)

In [10]:
# main parameters:

# desiredQuantity -> givenQuantity1, givenQuantity2, materialName(optional), gamma(optional), pinf(optional), cp (opt)
#       p         ->        T      ,      rho
#       T         ->        p      ,      rho
#       rho       ->        T      ,      p

main("temperature", 1.01325e5, 1117, "liquidwater", 2.35, 10e9, 4267)

desired quantity: temperature

gamma = 2.35
pinf = 1.0e10

output: 
p = 101325.0
T = 3652.268337354245
rho = 1117.0
e = 1.5584138283762723e7

ending main
